In [416]:
import numpy as np
import pandas as pd
import json
import os
import matplotlib as plt
import re

In [417]:
data_src = "./globalterrorismdb_0522dist.xlsx"

In [419]:
df = pd.read_excel(data_src)
df.head()

,Unnamed: 0,iyear,imonth,iday,extended,resolution,country_txt,region_txt,summary,multiple,...,divert,ransom,ransomamt,ransompaid,hostkidoutcome_txt,INT_LOG,INT_IDEO,INT_MISC,nkillvictams,ransom_paid_text
0,0,2000,1,1,0,NaT,Namibia,Sub-Saharan Africa,"In the first of two related incidents, Nationa...",Yes,...,NaN,Unknown,0.0,0.0,Unknown,Yes,Yes,No,0,"No Ransom, No Payment"
1,1,2000,1,1,1,NaT,Namibia,Sub-Saharan Africa,"In the second of two related incidents, the Na...",Yes,...,NaN,No,0.0,0.0,Unknown,Yes,Yes,No,0,"No Ransom, No Payment"
2,2,2000,1,1,0,NaT,Kosovo,Eastern Europe,"A Romany (gypsy) house in Pec, Kosovo, was bom...",No,...,NaN,Unknown,0.0,0.0,Unknown,Unknown,Unknown,Yes,0,"No Ransom, No Payment"
3,3,2000,1,1,0,NaT,Bangladesh,South Asia,"In a series of related incidents, numerous bom...",Yes,...,NaN,Unknown,0.0,0.0,Unknown,Unknown,Unknown,No,-99,"No Ransom, No Payment"
4,4,2000,1,1,0,NaT,India,South Asia,An Oil and Natural Gas Commission (ONGC) pipel...,No,...,NaN,Unknown,0.0,0.0,Unknown,No,No,No,0,"No Ransom, No Payment"


In [450]:
df.shape

(209706, 135)

In [421]:
for i, c in enumerate(df.columns):
    print(f"{i}. {c}")

0. Unnamed: 0
1. iyear
2. imonth
3. iday
4. extended
5. resolution
6. country_txt
7. region_txt
8. summary
9. multiple
10. success
11. suicide
12. attacktype1_txt
13. attacktype2_txt
14. attacktype3_txt
15. targtype1_txt
16. targtype2_txt
17. targtype3_txt
18. individual
19. claimed
20. compclaim
21. weaptype1_txt
22. weaptype2_txt
23. weaptype3_txt
24. weaptype4_txt
25. nkill
26. nkillter
27. nwound
28. property
29. ishostkid
30. divert
31. ransom
32. ransomamt
33. ransompaid
34. hostkidoutcome_txt
35. INT_LOG
36. INT_IDEO
37. INT_MISC
38. nkillvictams
39. ransom_paid_text


## Columns
0. eventid: ID 12-digit Event ID system - yyyymmdd + sequential case number for the given day
1. iyear: Year
2. imonth: Month
3. iday: Day
4. approxdate: Whenever the exact date of the incident is not known or remains unclear, this field is used to record the approximate date of the incident
5. extended: The duration of an incident extended more than 24 hours.
6. resolution: This field only applies if “Extended Incident?” is “Yes” and records the date in which the incident was resolved
7. country
8. country_txt
9. region
10. region_txt
11. provstate
12. city
13. latitude
14. longitude
15. specificity
16. vicinity
17. location
18. summary: A brief narrative summary of the incident
19. crit1: Criterion 1: POLITICAL, ECONOMIC, RELIGIOUS, OR SOCIAL GOAL
20. crit2: Criterion 2: INTENTION TO INTIMIDATE OR PUBLICIZE TO LARGER AUDIENCE
21. crit3: Criterion 3: OUTSIDE INTERNATIONAL HUMANITARIAN LAW
22. doubtterr: uncertainty whether an incident meets all of the criteria for inclusion
23. alternative: applies to cases coded as “Yes” for “Doubt Terrorism Proper?”.  This variable identifies the most likely categorization of the incident other than terrorism
24. alternative_txt
25. multiple: several attacks are connected
134. related: multi-part incident the GTD IDs of the related incidents are listed here
26. success
27. suicide
28. 33. attacktype1; attacktype1_txt: general method of attack and often reflects the broad class of tactics used
34. 57. targtype1; targtype1_txt; targsubtype1; targsubtype1_txt; corp1; natlty1; natlty1_txt
58. 67. gname: Perpetrator Group Name
64. motive: 
68. individual: the attack was carried out by an individual or several individuals not known to be affiliated with a group or organization
69. nperps: number of perps
70. nperpcap: number of Captured perps
71. claimed: whether a group or person(s) claimed responsibility for the attack.
72. 79. claimmode; claimmode_txt: This records one of 10 modes used by claimants to claim responsibility
80. compclaim: more than one group claimed separate responsibility for the attack
81. 96. weaptype1; weaptype1_txt, weapsubtype1; weapsubtype1_txt
98. nkill: number of kills from perps and victams
99. nkillus: number of US citizans kills from perps and victams
100. nkillter: number of kills from perps only
101. nwound
102. nwoundus
104. property: damage to property
105. 106. propextent; propextent_txt
109. ishostkid: hostages or kidnapping
110. nhostkid: hostages or kidnapping number
114. divert: Country That Kidnappers/Hijackers Diverted To
116. ransom
123. hostkidoutcome_txt: Kidnapping/Hostage Outcome
130. INT_LOG: The attack was logistically international; the nationality of the perpetrator group differs from the location of the attack
131. INT_IDEO: The attack was ideologically international; the nationality of the perpetrator group differs from the nationality of the target
132. INT_MISC: the location of the attack differs from the nationality of the target

In [422]:
remove_rows = {"doubtterr":1, }

In [451]:
remove_columns = [
    "eventid", 
    "approxdate", 
    "crit1", 
    "crit2", 
    "crit3", 
    "doubtterr", 
    "alternative", 
    "alternative_txt", 
    "related", 
    "country", 
    "region", 
    "provstate", 
    "city", 
    "vicinity", 
    "latitude", 
    "longitude", 
    "specificity",
    "attacktype1",
    "attacktype2",
    "attacktype3",
    "weaptype1",
    "weaptype2",
    "weaptype3",
    "weaptype4",
    "weapsubtype1",
    "weapsubtype2",
    "weapsubtype3",
    "weapsubtype4",
    "weapsubtype1_txt",
    "weapsubtype2_txt",
    "weapsubtype3_txt",
    "weapsubtype4_txt",
    "weapdetail",
    "targtype1",
    "targsubtype1",
    "targsubtype1_txt",
    "corp1",
    "target1",
    "natlty1",
    "natlty1_txt",
    "targtype2",
    "targsubtype2",
    "corp2",
    "target2",
    "natlty2",
    "targsubtype2_txt",
    "natlty2_txt",
    "targtype3",
    "targsubtype3",
    "corp3",
    "target3",
    "natlty3",
    "targsubtype3_txt",
    "natlty3_txt",
    "gname",
    "gsubname",
    "gname2",
    "gsubname2",
    "gname3",
    "gsubname3",
    "guncertain1",
    "guncertain2",
    "guncertain3",
    "nperps",
    "nperpcap",
    "nperpcap",
    "claimmode",
    "claimmode_txt",
    "claim2",
    "claimmode2",
    "claimmode2_txt",
    "claim3",
    "claimmode3",
    "claimmode3_txt",
    "motive",
    "nkillus",
    "nwoundus",
    "nwoundte",
    "propextent",
    "propextent_txt",
    "propvalue",
    "propcomment",
    "nhostkid",
    "nhostkidus",
    "nhours",
    "ndays",
    "kidhijcountry",
    "ransompaidus",
    "ransomamtus",
    "ransomnote",
    "hostkidoutcome",
    "nreleased",
    "addnotes",
    "addnotes",
    "scite1",
    "scite2",
    "scite3",
    "dbsource",
    "INT_ANY",
    "location"
    ]

# Cleaning

1. Remove unconfirmed acts

In [452]:
dfc = df[df["doubtterr"] != 1].copy()

2. Drop unwanted columns

In [453]:
dfc.drop(columns=remove_columns, inplace=True)

In [454]:
dfc.columns

Index(['iyear', 'imonth', 'iday', 'extended', 'resolution', 'country_txt',
       'region_txt', 'summary', 'multiple', 'success', 'suicide',
       'attacktype1_txt', 'attacktype2_txt', 'attacktype3_txt',
       'targtype1_txt', 'targtype2_txt', 'targtype3_txt', 'individual',
       'claimed', 'compclaim', 'weaptype1_txt', 'weaptype2_txt',
       'weaptype3_txt', 'weaptype4_txt', 'nkill', 'nkillter', 'nwound',
       'property', 'ishostkid', 'divert', 'ransom', 'ransomamt', 'ransompaid',
       'hostkidoutcome_txt', 'INT_LOG', 'INT_IDEO', 'INT_MISC'],
      dtype='object')

In [455]:
dfc.shape

(174471, 37)

In [467]:
features = [
    "suicide",
    "success",
    "extended",
    "property",
    "hostkidoutcome_txt",
    "multiple",
    "individual",
    "claimed",
    "compclaim",
    "ishostkid",
    "ransom",
    "INT_LOG",
    "INT_IDEO",
    "INT_MISC",
]

In [468]:
for feat in features:
    print(f"{feat:-^30}")
    print(df[feat].value_counts(dropna=False))
    print("-"*30)


-----------suicide------------
0    202268
1      7438
Name: suicide, dtype: int64
------------------------------
-----------success------------
1    185302
0     24404
Name: success, dtype: int64
------------------------------
-----------extended-----------
0    198901
1     10805
Name: extended, dtype: int64
------------------------------
-----------property-----------
 1    102927
 0     78731
-9     28048
Name: property, dtype: int64
------------------------------
------hostkidoutcome_txt------
NaN                                               195615
Unknown                                             4798
Hostage(s) released by perpetrators                 3824
Hostage(s) killed (not during rescue attempt)       3209
Combination                                         1399
Successful Rescue                                    653
Hostage(s) escaped (not during rescue attempt)       187
Attempted Rescue                                      21
Name: hostkidoutcome_txt, dtype: int64
-

3. Change categorical 1,0 values to "yes" "no"

In [469]:
dfc["success"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
dfc["suicide"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
dfc["property"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
dfc["multiple"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "No"}, inplace=True)
dfc["individual"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "No"}, inplace=True)
dfc["claimed"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
dfc["compclaim"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "NA"}, inplace=True)
dfc["ishostkid"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
dfc["hostkidoutcome_txt"].replace({np.nan: "Unknown"}, inplace=True)
dfc["ransom"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
dfc["INT_LOG"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
dfc["INT_IDEO"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
dfc["INT_MISC"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)


4. Clean `comclaim` invalid values with `claimed`.  

In [470]:
dfc.loc[(dfc["claimed"] == "Yes"), ["compclaim"]].value_counts()

compclaim
NA           16307
Unknown       1758
No             833
Yes            410
dtype: int64

All unknown claimed should be NA comclaim

In [471]:
dfc.loc[(dfc["claimed"] == "Unknown"), ["compclaim"]] = "NA"

All "No"" claimed should be "NA" comclaim

In [472]:
dfc.loc[(dfc["claimed"] == "No"), ["compclaim"]] = "NA"

All claimed should either be "yes", "No", "Unkown"  
replace "NA" to "Unkown"

In [473]:
dfc.loc[(dfc["claimed"] == "Yes") & (dfc["compclaim"] == "NA"), ["compclaim"]] = "Unknown"

5. Clean `ransomamt` invalid values with `ransom`.  

In [474]:
dfc["ransom"].value_counts()

Unknown    107422
No          65711
Yes          1338
Name: ransom, dtype: int64

if `ransom` is "no", then `ransomamt` should always be "0"  

In [475]:
dfc.loc[( (dfc["ransom"] == "No") | (dfc["ransom"] == "Unknown") ) & (dfc["ransomamt"]>0), ["ransomamt"]] = 0

In [476]:
dfc.loc[( (dfc["ransom"] == "No") | (dfc["ransom"] == "Unknown") ) & (dfc["ransomamt"]<0), ["ransomamt"]]

,ransomamt
69412,-99.0
69432,-99.0
71542,-99.0
71998,-99.0
72276,-9.0
72346,-99.0
72441,-99.0
72475,-99.0
72477,-99.0
72478,-99.0


In [477]:
dfc.loc[( (dfc["ransom"] == "No") | (dfc["ransom"] == "Unknown") ) & (dfc["ransomamt"]<0), ["ransomamt"]] = 0

In [481]:
df.loc[df["ransompaid"]>0, ["ransom"]].value_counts()

ransom
1.0       250
dtype: int64

6. Replace numerical columns "-99" to 0

In [485]:
df.loc[df["ransomamt"]<0, ["ransomamt"]].value_counts()

ransomamt
-99.0        322
-9.0           3
dtype: int64

In [ ]:
dfc["ransomamt"].replace({np.nan: 0}, inplace=True)
dfc["ransompaid"].replace({np.nan: 0}, inplace=True)

7. remove date from summary

In [ ]:
def clean_str(text):
    if isinstance(text, str):
        prefix_date = r"^\d+\/\d+\/\d+:?\s?"
        return re.sub(prefix_date, "", text)
    else:
        return "No Information Available"

In [ ]:
sample_text = "01/01/2000: In the first of two related incidents, National Union for the Total Independence of Angola (UNITA) members attacked a tavern in the village of Muitjiku, Namibia. The owner of the tavern fired back at UNITA, injuring some of the rebels."

In [ ]:
clean_str(sample_text)

'In the first of two related incidents, National Union for the Total Independence of Angola (UNITA) members attacked a tavern in the village of Muitjiku, Namibia. The owner of the tavern fired back at UNITA, injuring some of the rebels.'

In [ ]:
dfc["summary"] = dfc["summary"].apply(lambda x: clean_str(x))

**Create new features**

8. Number of killed victams

In [528]:
dfc["nkillvictams"] = dfc["nkill"] - dfc["nkillter"]

In [529]:
dfc["nkillvictams"].value_counts(dropna=False)

0.0      61651
NaN      60338
1.0      24558
2.0       9778
3.0       4998
         ...  
142.0        1
106.0        1
287.0        1
315.0        1
207.0        1
Name: nkillvictams, Length: 157, dtype: int64

In [491]:
dfc.loc[dfc["nkillvictams"]<0, ["nkillvictams"]]

,nkillvictams


In [534]:
dfc.loc[dfc["nkillter"]>dfc["nkill"], ["nkillter"]]

,nkillter


In [533]:
dfc.loc[111]

iyear                                                              1970
imonth                                                                3
iday                                                                  5
extended                                                              0
resolution                                                          NaT
country_txt                                               United States
region_txt                                                North America
summary               3/5/1970: Unknown Puerto Rican Nationalists as...
multiple                                                             No
success                                                             Yes
suicide                                                              No
attacktype1_txt                                           Assassination
attacktype2_txt                                                     NaN
attacktype3_txt                                                 

In [492]:
dfc.loc[dfc["nkill"]<dfc["nkillter"], ["nkill"]]

,nkill


- Ransom paid compared to requested amount

In [521]:
def ransom_paid_txt(amount, paid):
    if ( (np.isnan(amount) and np.isnan(paid)) or 
         (np.isnan(amount) and paid == 0) or 
         (amount == 0 and np.isnan(paid)) or 
         (amount == paid == 0)):
        res = "No Ransom"
    elif amount == -99 or paid == -99 or np.isnan(paid) or np.isnan(amount):
        res = "Unknown"
    elif amount == 0 and paid > 0:
        res = "No Ransom, but Paid!"
    elif amount > 0 and paid == 0:
        res = "Ransom not paid"
    elif amount > paid:
        res = "Ransom paid partially"
    elif amount < paid:
        res = "Ransom paid with extra!"
    elif amount == paid:
        res = "Ransom paid exactly"
    else:
        res = "Unknown CASE"
    return res

In [522]:
dfc['ransompaid'].isna().any()

True

In [523]:
dfc["ransom_paid_text"] = dfc.apply(lambda x: ransom_paid_txt(x['ransomamt'], x['ransompaid']), axis=1)

In [524]:
dfc["ransom_paid_text"].value_counts()

No Ransom                  173407
Unknown                       707
Ransom not paid               164
Ransom paid exactly           102
Ransom paid partially          86
Ransom paid with extra!         5
Name: ransom_paid_text, dtype: int64

In [527]:
dfc[(dfc["ransom_paid_text"]=="Unknown") & (dfc["ransomamt"]>0)].iloc[1]

iyear                                                          1970
imonth                                                            3
iday                                                             31
extended                                                          1
resolution                                      1970-04-05 00:00:00
country_txt                                               Guatemala
region_txt                              Central America & Caribbean
summary                                                         NaN
multiple                                                         No
success                                                         Yes
suicide                                                          No
attacktype1_txt                         Hostage Taking (Kidnapping)
attacktype2_txt                                                 NaN
attacktype3_txt                                                 NaN
targtype1_txt                               Gove

In [ ]:
df[(df["ransomamt"] != 0) & (df["ransomamt"] == df["ransompaid"])].iloc[0][["ransomamt", "ransompaid"]]

ransomamt     110000.0
ransompaid    110000.0
Name: 27, dtype: object

In [ ]:
dfc.loc[27, ["ransomamt", "ransompaid", "ransom_paid_text"]]

ransomamt                      110000.0
ransompaid                     110000.0
ransom_paid_text    Ransom paid exactly
Name: 27, dtype: object

8. clean `hostkidoutcome_txt`

In [430]:
dfc[dfc["ishostkid"] != "Yes"]["hostkidoutcome_txt"].value_counts()

Unknown                                           104746
Hostage(s) escaped (not during rescue attempt)         3
Name: hostkidoutcome_txt, dtype: int64

In [431]:
dfc.loc[dfc["ishostkid"] != "Yes", ["hostkidoutcome_txt"]] = "NA"

In [432]:
dfc[dfc["ishostkid"] != "Yes"]["hostkidoutcome_txt"].value_counts()

Unknown    104749
Name: hostkidoutcome_txt, dtype: int64

# Save

In [71]:
dfc_pre2k = dfc[dfc["iyear"] < 2000].reset_index(drop=True, inplace=False)
dfc_post2k = dfc[dfc["iyear"] >= 2000].reset_index(drop=True, inplace=False)

In [72]:

dfc_post2k.to_excel("cleaned.xlsx")

# Abstraction Function

In [442]:
def load_excel_df(path: str) -> pd.DataFrame:
    return pd.read_excel(path)

In [538]:
def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean a dataframe and return cleaned version
    """
    
    _remove_columns = [
    "eventid", 
    "approxdate",
    "resolution",
    "crit1", 
    "crit2", 
    "crit3", 
    "doubtterr", 
    "alternative", 
    "alternative_txt", 
    "related", 
    "country", 
    "region", 
    "provstate", 
    "city", 
    "vicinity", 
    "location",
    "latitude", 
    "longitude", 
    "specificity",
    "attacktype1",
    "attacktype2",
    "attacktype3",
    "weaptype1",
    "weaptype2",
    "weaptype3",
    "weaptype4",
    "weapsubtype1",
    "weapsubtype2",
    "weapsubtype3",
    "weapsubtype4",
    "weapsubtype1_txt",
    "weapsubtype2_txt",
    "weapsubtype3_txt",
    "weapsubtype4_txt",
    "weapdetail",
    "targtype1",
    "targsubtype1",
    "targsubtype1_txt",
    "corp1",
    "target1",
    "natlty1",
    "natlty1_txt",
    "targtype2",
    "targsubtype2",
    "corp2",
    "target2",
    "natlty2",
    "targsubtype2_txt",
    "natlty2_txt",
    "targtype3",
    "targsubtype3",
    "corp3",
    "target3",
    "natlty3",
    "targsubtype3_txt",
    "natlty3_txt",
    "gname",
    "gsubname",
    "gname2",
    "gsubname2",
    "gname3",
    "gsubname3",
    "guncertain1",
    "guncertain2",
    "guncertain3",
    "nperps",
    "nperpcap",
    "nperpcap",
    "claimmode",
    "claimmode_txt",
    "claim2",
    "claimmode2",
    "claimmode2_txt",
    "claim3",
    "claimmode3",
    "claimmode3_txt",
    "motive",
    "nkillus",
    "nwoundus",
    "nwoundte",
    "propextent",
    "propextent_txt",
    "propvalue",
    "propcomment",
    "nhostkid",
    "nhostkidus",
    "nhours",
    "ndays",
    "kidhijcountry",
    "ransompaidus",
    "ransomamtus",
    "ransomnote",
    "hostkidoutcome",
    "nreleased",
    "addnotes",
    "addnotes",
    "scite1",
    "scite2",
    "scite3",
    "dbsource",
    "INT_ANY"
    ]
    # 1. Remove unconfirmed acts
    dfc = df[df["doubtterr"] != 1].copy()
    
    # 2. Drop unwanted columns
    dfc.drop(columns=_remove_columns, inplace=True)

    # 3. Change numerical values to "yes" "no" 
    dfc["success"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
    dfc["suicide"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
    dfc["property"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
    dfc["multiple"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "No"}, inplace=True)
    dfc["individual"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "No"}, inplace=True)
    dfc["claimed"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
    dfc["compclaim"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "NA"}, inplace=True)
    dfc["ishostkid"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
    dfc["hostkidoutcome_txt"].replace({np.nan: "Unknown"}, inplace=True)
    dfc["ransom"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
    dfc["INT_LOG"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
    dfc["INT_IDEO"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)
    dfc["INT_MISC"].replace({1: "Yes", 0: "No", -9: "Unknown", np.nan: "Unknown"}, inplace=True)

    # 4. Clean `comclaim` invalid values when compared to `claimed`.  
        # - All unknown claimed should be NA comclaim
        # - All "No"" claimed should be "NA" comclaim
        # - All claimed should either be "yes", "No", "Unkown" , replace "NA" to "Unkown"
    dfc.loc[(dfc["claimed"] == "Unknown"), ["compclaim"]] = "NA"
    dfc.loc[(dfc["claimed"] == "No"), ["compclaim"]] = "NA"
    dfc.loc[(dfc["claimed"] == "Yes") & (dfc["compclaim"] == "NA"), ["compclaim"]] = "Unknown"

    # 5. Clean `ransomamt` invalid values with `ransom`.  
        # if `ransom` is "no", then `ransomamt` should always be "0"
    dfc.loc[(dfc["ransom"] == "No") | (dfc["ransom"] == "Unknown"), ["ransomamt"]] = 0

    # # 6. Replace "NaN" to 0
    # dfc["ransomamt"].replace({np.nan: 0}, inplace=True)
    # dfc["ransompaid"].replace({np.nan: 0}, inplace=True)

    # 7. clean summary
    def _clean_str(text):
        if isinstance(text, str):
            prefix_date = r"^\d+\/\d+\/\d+:?\s?"
            return re.sub(prefix_date, "", text)
        else:
            return "No Information Available"
    dfc["summary"] = dfc["summary"].apply(lambda x: _clean_str(x))
    
    # 8. Create new features - Number of killed victams
    dfc["nkillvictams"] = dfc["nkill"] - dfc["nkillter"]
    
    # Create new features - Ransom paid compared to requested amount
    def _ransom_paid_txt(amount, paid):
        if ( (np.isnan(amount) and np.isnan(paid)) or 
            (np.isnan(amount) and paid == 0) or 
            (amount == 0 and np.isnan(paid)) or 
            (amount == paid == 0)):
            res = "No Ransom"
        elif amount == -99 or paid == -99 or np.isnan(paid) or np.isnan(amount):
            res = "Unknown"
        elif amount == 0 and paid > 0:
            res = "No Ransom, but Paid!"
        elif amount > 0 and paid == 0:
            res = "Ransom not paid"
        elif amount > paid:
            res = "Ransom paid partially"
        elif amount < paid:
            res = "Ransom paid with extra!"
        elif amount == paid:
            res = "Ransom paid exactly"
        else:
            res = "Unknown CASE"
        return res
    
    dfc["ransom_paid_text"] = dfc.apply(lambda x: _ransom_paid_txt(x['ransomamt'], x['ransompaid']), axis=1)
    
    # clean -99 from ransom, ransompaid, nkill, nkillter
    dfc["ransomamt"].replace({-99: np.nan, -9: np.nan}, inplace=True)
    dfc["ransompaid"].replace({-99: np.nan, -9: np.nan}, inplace=True)
    dfc["nkill"].replace({-99: np.nan, -9: np.nan}, inplace=True)
    dfc["nkillter"].replace({-99: np.nan, -9: np.nan}, inplace=True)

    # clean hostkidoutcome_txt
    dfc.loc[dfc["ishostkid"] != "Yes", ["hostkidoutcome_txt"]] = "NA"

    return dfc

In [536]:
def save_split(
    df: pd.DataFrame, 
    split_year : int = 2000,
    save_split : str = "post",
    )-> None: 
    """
    Save DataFrame into excel.  

    Args:  
    - df: DataFrame to save.  
    - split_year: year to split the dataframe on, use 0 to not split  
    - save_split: which split to save, use one of ["post", "pre", "both"]  
    """
    
    dfc_pre = df[df["iyear"] < split_year].reset_index(drop=True, inplace=False)
    dfc_pst = df[df["iyear"] >= split_year].reset_index(drop=True, inplace=False)

    # if save == "post":
    #     dfc_pst.to_excel(f"cleaned_post_{split_year}.xlsx")
    # elif save == "pre":
    #     dfc_pre.to_excel(f"cleaned_post_{split_year}.xlsx")
    # else:
    #     dfc_pst.to_excel(f"cleaned_post_{split_year}.xlsx")
    #     dfc_pre.to_excel(f"cleaned_post_{split_year}.xlsx")

    if save_split != "post":
        dfc_pre.to_excel(f"cleaned_pre_{split_year}.xlsx")
    
    if save_split != "pre":
        dfc_pst.to_excel(f"cleaned_post_{split_year}.xlsx")




In [445]:
path = "./globalterrorismdb_0522dist.xlsx"

In [446]:
df = load_excel_df(path)

In [539]:
dfc = clean_df(df)

In [544]:
dfc.tail(2).iloc[1]


iyear                                                              2020
imonth                                                               12
iday                                                                 31
extended                                                              0
country_txt                                                 Afghanistan
region_txt                                                   South Asia
summary               Assailants shot and killed a civil society act...
multiple                                                             No
success                                                             Yes
suicide                                                              No
attacktype1_txt                                           Armed Assault
attacktype2_txt                                                     NaN
attacktype3_txt                                                     NaN
targtype1_txt                               Private Citizens & P

In [448]:
save_split(dfc, 2000, "post")